# Infering values


In [1]:
import json

data = json.load(open("../data/output.json"))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(data[2045]["img"])
plt.axis('off')  # Hide axis
plt.title("64x64 RGB Image")
plt.show()

## Building Training dataset

In [2]:
res = []

DEFAULT_YELLOW = [255, 170, 0]
YELLOW_MAX_DISTANCE = 50

import math
def distance(p1, p2):
    return math.sqrt(
        (p2[0] - p1[0]) ** 2 +
        (p2[1] - p1[1]) ** 2 +
        (p2[2] - p1[2]) ** 2
    )

# distance(DEFAULT_YELLOW, [0, 32, 0])

def minimum_distance_idx(lidar):
    min_i = 0
    for i in range(1,len(lidar)):
        if lidar[i] < lidar[min_i]:
            min_i = i
    return min_i

for i in range(len(data)):
    idx = minimum_distance_idx(data[i]["lidar"])

    # remove infs
    for lidar_idx in range(len(data[i]['lidar'])):
        data[i]['lidar'][lidar_idx] = min(5, data[i]['lidar'][lidar_idx])


    angle = (idx/128)*90 # regra de tres para achar o angulo entre 0 e 90 graus

    pixel_idx = int((idx/128)*64) # regra de tres para achar o pixel correspondente entre o pixel 0 e 64
    middle_pixel = data[i]["img"][32][pixel_idx]
    yellow_distance = distance(DEFAULT_YELLOW, middle_pixel)

    res.append({
        "img": data[i]["img"],
        "lidar": data[i]["lidar"],
        "dist": data[i]['lidar'][idx],
        "angle": angle,
        "yellow_distance": yellow_distance
    })

import json
with open("../data/training_data.json", 'w') as f:
    json.dump(res,f)

## bayesian modelling

In [ ]:
def get_values_from_cnn():
    return 1,2,3

class BayesianNetwork:
    # left = -1 , forward = 0, right = 1

    def __init__(self):
        # (obstacle_detected, target_visible, direction, action) -> success
        self.success_cpt = {}
        for obstacle_detected in (True, False):
            for target_visible in (True, False):
                for direction in range(-1,2):
                    for action in range(-1,2):
                        prob = 0.5

                        # if obstacle was detected and it is not the target, go to the opposite direction
                        if obstacle_detected:
                            if not target_visible:
                                if direction == action:
                                    prob = 0.1
                                elif direction == -action:
                                    prob = 0.8
                            else:
                                if direction == action:
                                    prob = 0.9
                                elif direction == -action:
                                    prob = 0.1
                        else:
                            if action == 0:
                                prob = 0.7

                        self.success_cpt[(obstacle_detected, target_visible, direction, action)] = prob
                                
    def obstacle_detected(self, dist):
        return dist < 0.4

    def direction(self,angle):
        if angle <= 30:
            return -1
        elif angle <= 60:
            return 0
        else:
            return 1

    def target_visible(self,is_yellow):
        YELLOW_MAX_DISTANCE = 100
        return is_yellow < YELLOW_MAX_DISTANCE

    def next_action(self, dist,angle,is_yellow):
        obstacle_detected = self.obstacle_detected(dist)
        direction = self.direction(angle)
        target_visible = self.target_visible(is_yellow)
        
        mx_action = [None, 0]
        for action in range(-1,2):
            success_probability = self.success_cpt[(obstacle_detected, target_visible, direction, action)]
            if success_probability > mx_action[1]:
                mx_action = [action, success_probability]
        return mx_action[0]
            
# perform robot actions:
class RobotActionController:
    def perform_action(self,action):
        ACTION_MAP = {
            "left": self.turn_left,
            "right": self.turn_right,
            "forward": self.move_forward
        }
        return ACTION_MAP[action]()

    def turn_left(self):
        pass
    def turn_right(self):
        pass
    def move_forward(self):
        pass

robot_controller = RobotActionController()
bayesian_network = BayesianNetwork()
while True:
    dist, angle, is_yellow = get_values_from_cnn()
    if is_yellow and dist < 0.3:
        break
    next_action = bayesian_network.next_action(dist,angle,is_yellow)
    robot_controller.perform_action(next_action)